In [278]:
# 추후 진행... 현황을 기준으로 해서 값을 고정할까 싶다... 그러면 보궐선거 전에는 비어 있을 텐데... 그럴 떄는 따로?

In [3]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

In [4]:
from datetime import datetime,timedelta

In [280]:
for start in pd.date_range('2020-06-01','2023-09-26'):
    ## 사용할 기간을 지정합니다.
    end = start + timedelta(90)

    ## 데이터 불러오기
    with open('data/raw2.pickle', 'rb') as f:
        raw2 = pickle.load(f)

    ## 데이터 복사
    raw3 = raw2.copy()

    ## 사용할 기간에 따라 데이터를 추출합니다

    raw3 = raw3[raw3['date'].between(start, end)]

    ## 데이터 구분
    ##[이름,당명,한자명] 형식인 데이터를 각각 하나의 열로 구분합니다. 한자명은 사용하지 않습니다.

    raw3['main_name'] = [x[0] for x in raw3['main']]
    raw3['main_party'] = [x[1] for x in raw3['main']]

    raw3 = raw3.explode('members')

    raw3['member_name'] = [x[0] for x in raw3['members']]
    raw3['member_party'] = [x[1] for x in raw3['members']]

    raw3 = raw3.drop(['main','members'],axis = 1)

    ## 당명 변경
    raw3['main_party'] = [x.replace('미래통합당','국민의힘') for x in raw3['main_party']]
    raw3['member_party'] = [x.replace('미래통합당','국민의힘') for x in raw3['member_party']]

    ## 공백 제거
    raw3['main_name'] = [x.replace('\n','') for x in raw3['main_name']]
    raw3['main_name'] = [x.replace('\t','') for x in raw3['main_name']]

    raw3['main_party'] = [x.replace('\n','') for x in raw3['main_party']]
    raw3['main_party'] = [x.replace('\t','') for x in raw3['main_party']]

    raw3['member_name'] = [x.replace('\n','') for x in raw3['member_name']]
    raw3['member_name'] = [x.replace('\t','') for x in raw3['member_name']]

    raw3['member_party'] = [x.replace('\n','') for x in raw3['member_party']]
    raw3['member_party'] = [x.replace('\t','') for x in raw3['member_party']]

    ## 매트릭스 인덱스 이름 설정 기본 : 한글명 + 당명
    raw3['main'] = raw3['main_name'] + ' ' + raw3['main_party']
    raw3['member'] = raw3['member_name'] + ' ' + raw3['member_party']

    matrix = raw3.pivot_table(index="main", columns="member", aggfunc="size", fill_value=0)

    # 전처리된 자료 저장.    
    with open('data/matrix_{}_{}.pickle'.format(str(start)[:10],str(end)[:10]), 'wb') as f:
        pickle.dump(matrix , f, pickle.HIGHEST_PROTOCOL)


In [177]:
## 데이터 불러오기
with open('data/raw2.pickle', 'rb') as f:
    raw2 = pickle.load(f)

In [178]:
raw3 = raw2.copy()

In [179]:
raw3['main_name'] = [x[0] for x in raw3['main']]
raw3['main_party'] = [x[1] for x in raw3['main']]

raw3 = raw3.explode('members')

raw3['member_name'] = [x[0] for x in raw3['members']]
raw3['member_party'] = [x[1] for x in raw3['members']]

## 당명 변경
raw3['main_party'] = [x.replace('미래통합당','국민의힘') for x in raw3['main_party']]
raw3['member_party'] = [x.replace('미래통합당','국민의힘') for x in raw3['member_party']]

## 공백 제거
raw3['main_name'] = [x.replace('\n','') for x in raw3['main_name']]
raw3['main_name'] = [x.replace('\t','') for x in raw3['main_name']]

raw3['main_party'] = [x.replace('\n','') for x in raw3['main_party']]
raw3['main_party'] = [x.replace('\t','') for x in raw3['main_party']]

raw3['member_name'] = [x.replace('\n','') for x in raw3['member_name']]
raw3['member_name'] = [x.replace('\t','') for x in raw3['member_name']]

raw3['member_party'] = [x.replace('\n','') for x in raw3['member_party']]
raw3['member_party'] = [x.replace('\t','') for x in raw3['member_party']]

'''
## 매트릭스 인덱스 이름 설정 기본 : 한글명 + 당명
raw3['main'] = raw3['main_name'] + ' ' + raw3['main_party']
raw3['member'] = raw3['member_name'] + ' ' + raw3['member_party']
'''
raw3 = raw3.reset_index(drop = False)

"\n## 매트릭스 인덱스 이름 설정 기본 : 한글명 + 당명\nraw3['main'] = raw3['main_name'] + ' ' + raw3['main_party']\nraw3['member'] = raw3['member_name'] + ' ' + raw3['member_party']\n"

In [204]:
# 동명이인 처리 : 경우의 수가 많아서 그냥 서치!
# 국회의원 현환.https://www.assembly.go.kr/portal/cnts/cntsCont/dataA.do?cntsDivCd=NAAS&menuNo=600137
# 일일이 다 바꿔줌 ㅠㅠㅠ

for idx,x in enumerate(raw3['main'].values):
    if x[0] == '김병욱' and x[1] == '더불어민주당':
        raw3.loc[idx,'main_name'] = '김병욱(더)'
    elif x[0] == '김병욱' and x[1] == '국민의힘':
        raw3.loc[idx,'main_name'] = '김병욱(국)'
    elif x[0] == '김병욱' and x[1] == '미래통합당':
        raw3.loc[idx,'main_name'] = '김병욱(국)'
    elif x[0] == '김병욱' and x[1] == '무소속':
        raw3.loc[idx,'main_name'] = '김병욱(국)'
    else:
        pass
    
for idx,x in enumerate(raw3['members'].values):
    if x[0] == '김병욱' and x[1] == '더불어민주당':
        raw3.loc[idx,'member_name'] = '김병욱(더)'
    elif x[0] == '김병욱' and x[1] == '국민의힘':
        raw3.loc[idx,'member_name'] = '김병욱(국)'
    elif x[0] == '김병욱' and x[1] == '미래통합당':
        raw3.loc[idx,'member_name'] = '김병욱(국)'
    elif x[0] == '김병욱' and x[1] == '무소속':
        raw3.loc[idx,'member_name'] = '김병욱(국)'
    else:
        pass
    
## 하....main_ member_ 로 다시 짜야함... 아래 보면 알겠듯이 공백이 문제임....ㅅㅂ

In [205]:
raw3[raw3['member_name'] == '김병욱']

,index,code,date,members,main,law,main_name,main_party,member_name,member_party
67183,5507,PRC_L2K2N1D1L2P8Q1I1O1Z3F1Q7N4G8Z1,2022-11-28,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[박홍근, 더불어민주당, 朴洪根]",[2118497]반인권적 국가범죄의 시효 등에 관한 특례법안(박홍근의원 외 168인),박홍근,더불어민주당,김병욱,더불어민주당
70288,5748,PRC_T2V2L1W1Z1O4L1M1Q2O3S2F3J2Y0Z1,2022-11-14,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[박범계, 더불어민주당, 朴範界]",[2118238]감사원법 일부개정법률안(박범계의원 외 168인),박범계,더불어민주당,김병욱,더불어민주당
71207,5813,PRC_D2R2R1Y1Q0F8Z1U4R5P0E0G8M4R6F9,2022-11-08,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[김성환, 더불어민주당, 金星煥]",[2118165]하도급거래 공정화에 관한 법률 일부개정법률안(김성환의원 외 168인),김성환,더불어민주당,김병욱,더불어민주당
71386,5815,PRC_R2C2A1C1V0T8Y1J4Z3O5L1X8X4K1B7,2022-11-08,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[김성환, 더불어민주당, 金星煥]",[2118163]대·중소기업 상생협력 촉진에 관한 법률 일부개정법률안(김성환의원 외...,김성환,더불어민주당,김병욱,더불어민주당
81044,6571,PRC_U2O2A0P9P0L7I1K4N4C2V3T7O3J5O1,2022-09-07,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[박홍근, 더불어민주당, 朴洪根]","[2117292]윤석열 대통령 배우자 김건희의 주가조작, 허위경력, 뇌물성 후원 사...",박홍근,더불어민주당,김병욱,더불어민주당
101227,8252,PRC_O2M2Q0E4O2C7R1E8R2I6I4Q4C2Y5E6,2022-04-27,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[김종민, 더불어민주당, 金鐘民]",[2115428]정보통신망 이용촉진 및 정보보호 등에 관한 법률 일부개정법률안(김종...,김종민,더불어민주당,김병욱,더불어민주당
101398,8253,PRC_D2C2B0Z4X2Y7A1H8P0Q8D2D0V8J2D4,2022-04-27,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[정필모, 더불어민주당, 鄭必模]",[2115427]방송법 일부개정법률안(정필모의원 외 170인),정필모,더불어민주당,김병욱,더불어민주당
101583,8255,PRC_S2Q2Y0M4H2Q7E1N7F1P9Q1W1P2R3X8,2022-04-27,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[정필모, 더불어민주당, 鄭必模]",[2115424]한국교육방송공사법 일부개정법률안(정필모의원 외 170인),정필모,더불어민주당,김병욱,더불어민주당
101754,8256,PRC_P2Q2Z0U4S2B7R1R7S1L5X5U0H0M8N0,2022-04-27,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[정필모, 더불어민주당, 鄭必模]",[2115423]방송문화진흥회법 일부개정법률안(정필모의원 외 170인),정필모,더불어민주당,김병욱,더불어민주당
101937,8258,PRC_Y2F2P0C4U2B7H1M6B3G8C2U5Y5I6S9,2022-04-27,"[\n\t\t\t\t\t\n\t\t\t\t\t김병욱, 더불어민주당, 金炳旭, \n...","[정필모, 더불어민주당, 鄭必模]",[2115421]방송통신위원회의 설치 및 운영에 관한 법률 일부개정법률안(정필모의원...,정필모,더불어민주당,김병욱,더불어민주당


In [ ]:
## 사용하지 않을 열은 제거
raw3 = raw3.drop(['main','members'],axis = 1)

In [63]:
# 당 변경 기록 데이터 -> 이것도 동명이인 문제가 해결되면 다시 작성
to_dates = []
from_dates = []
partys = []
names = []

for member in list(set(raw3['main'])):
    to_date = raw3[raw3['main'] == member]['date'].values[0]
    from_date = raw3[raw3['main'] == member]['date'].values[-1]
    member = member.split(' ')
    name = member[0]
    party = member[1]
    
    to_dates.append(to_date)
    from_dates.append(from_date)
    partys.append(party)
    names.append(name)

In [64]:
committeesPartyHistory = pd.DataFrame({'party':partys,'name':names,'from_date':from_dates,'to_date':to_dates})